In [55]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![Climate change](https://assets.nrdc.org/sites/default/files/styles/header_background--retina/public/personalaction-kidsclimatechange-2400x1600.jpg?itok=PeSlNGgX)

## Business Problem

Climate change is a globally relevant, urgent, and multi-faceted issue heavily impacted by energy policy and infrastructure. Addressing climate change involves mitigation (i.e. mitigating greenhouse gas emissions) and adaptation (i.e. preparing for unavoidable consequences). Mitigation of GHG emissions requires changes to electricity systems, transportation, buildings, industry, and land use.

According to a report issued by the International Energy Agency (IEA), the lifecycle of buildings from construction to demolition were responsible for 37% of global energy-related and process-related CO2 emissions in 2020. Yet it is possible to drastically reduce the energy consumption of buildings by a combination of easy-to-implement fixes and state-of-the-art strategies. For example, retrofitted buildings can reduce heating and cooling energy requirements by 50-90 percent. Many of these energy efficiency measures also result in overall cost savings and yield other benefits, such as cleaner air for occupants. This potential can be achieved while maintaining the services that buildings provide.

In [56]:
#import necessary liabries for data exploration and machine learning
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

import itertools
import math
import time
import pickle

import statsmodels.api as sm
import category_encoders as ce
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from scipy import stats
from sklearn import svm
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler,PolynomialFeatures 
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVR
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import HuberRegressor

I will be bringing in one of the functions I used for one of my projects, it is going to be useful throughout the training process to evaluate the model.

In [57]:
#Function to assist with evaluating the models
def evaluate (estimator, X_train,y_train, X_val, y_val):
    """
    Evaluate the amount of error between the model's predictions 
    and actiual values for both train and test set
    
    Parameters:
    y_train - array like, actual values for 'price' 
    train_preds - array like predicted values for 'price'
    y_val/test -array like actual values for 'price'
    val_preds/test - array like predicted values for 'price'
    returns:
    None
    """
    
   #predict the target on the training and validation data
    train_preds = estimator.predict(X_train)
    val_preds = estimator.predict(X_val)
    
    print("Scores")
    
    print(f"Train score: {train_preds}") #training score
    print(f"validation score: {val_preds}") #validation/Test score
    
    print("Rsquared:")

    print(f"Train R2: {r2_score(y_train,train_preds):.4f}") #rsquare metrics for train
    print(f"Validation R2: {r2_score(y_val,val_preds):.4f}") #rsquare metrics for val/test

    print("-----")

    print("Root Mean squared Error:")

    print(f"Train RMSE: {mean_squared_error(y_train,train_preds,squared = False):.2f}") #rmse metrics for train
    print(f"Validation RMSE: {mean_squared_error(y_val,val_preds, squared = False):.2f}") #rmse metrics for val/test

    print("-----")

    print(" Mean absolute Error:")

    print(f"Train MAE: {mean_absolute_error(y_train,train_preds):.2f}") #MAE metrics for train
    print(f"Validation MAE: {mean_absolute_error(y_val,val_preds):.2f}") #MAE metrics for val/test
    
    #calculate the residual
    train_residual = y_train - train_preds #residual for train
    val_residual = y_val - val_preds #residual for val / test
    
    #QQplot to check the normality of our 
    sm.qqplot(val_residual,line ='45',fit =True) 

## Data Understanding

The WiDS Datathon 2022 focuses on a prediction task involving roughly 100k observations of building energy usage records collected over 7 years and a number of states within the United States. The dataset consists of building characteristics (e.g. floor area, facility type etc), weather data for the location of the building (e.g. annual average temperature, annual total precipitation etc) as well as the energy usage for the building and the given year, measured as Site Energy Usage Intensity (Site EUI). Each row in the data corresponds to the a single building observed in a given year. Your task is to predict the Site EUI for each row, given the characteristics of the building and the weather data for the location of the building.

You are provided with two datasets: (1) the training dataset where the observed values of the Site EUI for each row is provided and (2) the test dataset where we withhold the observed values of the Site EUI for each row. To participate in the Datathon, you will submit a solution file containing the predicted Site EUI values for each row in the test dataset. The predicted values you submit will be compared against the observed Site EUI values for the test dataset and this will determine your standing on the Leaderboard during the competition as well as your final standing when the competition closes.


**Features**

- id: building id

- Year_Factor: anonymized year in which the weather and energy usage factors were observed

- State_Factor: anonymized state in which the building is located

- building_class: building classification

- facility_type: building usage type

- floor_area: floor area (in square feet) of the building

- year_built: year in which the building was constructed

- energy_star_rating: the energy star rating of the building

- ELEVATION: elevation of the building location

- january_min_temp: minimum temperature in January (in Fahrenheit) at the location of the building

- january_avg_temp: average temperature in January (in Fahrenheit) at the location of the building

- january_max_temp: maximum temperature in January (in Fahrenheit) at the location of the building

- cooling_degree_days: cooling degree day for a given day is the number of degrees where the daily average temperature exceeds 65 degrees Fahrenheit. Each month is summed to produce an annual total at the location of the building.

- heating_degree_days: heating degree day for a given day is the number of degrees where the daily average temperature falls under 65 degrees Fahrenheit. Each month is summed to produce an annual total at the location of the building.

- precipitation_inches: annual precipitation in inches at the location of the building

- snowfall_inches: annual snowfall in inches at the location of the building

- snowdepth_inches: annual snow depth in inches at the location of the building

- avg_temp: average temperature over a year at the location of the building

- days_below_30F: total number of days below 30 degrees Fahrenheit at the location of the building

- days_below_20F: total number of days below 20 degrees Fahrenheit at the location of the building

- days_below_10F: total number of days below 10 degrees Fahrenheit at the location of the building

- days_below_0F: total number of days below 0 degrees Fahrenheit at the location of the building

- days_above_80F: total number of days above 80 degrees Fahrenheit at the location of the building

- days_above_90F: total number of days above 90 degrees Fahrenheit at the location of the building

- days_above_100F: total number of days above 100 degrees Fahrenheit at the location of the building

- days_above_110F: total number of days above 110 degrees Fahrenheit at the location of the building

- direction_max_wind_speed: wind direction for maximum wind speed at the location of the building. Given in 360-degree compass point directions (e.g. 360 = north, 180 = south, etc.).

- direction_peak_wind_speed: wind direction for peak wind gust speed at the location of the building. Given in 360-degree compass point directions (e.g. 360 = north, 180 = south, etc.).

- max_wind_speed: maximum wind speed at the location of the building

- days_with_fog: number of days with fog at the location of the building

 **Target**
- site_eui: Site Energy Usage Intensity is the amount of heat and electricity consumed by a building as reflected in utility bills

Data has already been split so I will only be working with the train dataset and then vote for the best model to predict the test dataset.

In [58]:
#read the train data
train_data = pd.read_csv("/kaggle/input/widsdatathon2022/train.csv")
test_data = pd.read_csv("/kaggle/input/widsdatathon2022/test.csv")

In [59]:
#view the first ten rows of the data
train_data.head(10)

In [60]:
#check the data info
train_data.info()

In [61]:
#let's view the shape of the data
train_data.shape

In [62]:
#view the missing values per features in percentage
train_data.isna().mean().sort_values().plot(
    kind="bar", figsize=(25, 10),
    title="Percentage of missing values per feature",
    ylabel="Ratio of missing values per feature");

There are 55.2% missing values in direction_peak_wind_speed columns that are missing, 54.23% values in max_wind_speed and direction_max_wind_speed. Also 60.1% missing values from the days_with_fog columns. Those features have more than 50% missing values. So let's go ahead and drop any features with more than 15% missing values. 

In [63]:
#drop features with more than 15% missing values
train_data = train_data.dropna(thresh = train_data.shape[0] *0.85, axis=1)
train_data.shape

In [64]:
#check if there is duplicated values
train_data.duplicated().sum()

In [65]:
#check the remaining missing values.
train_data.isnull().sum().sum()

This is great we can go ahead and fill the values since it is leass than 15%.

In [66]:
#fill all the nan values with zero
#check if there is still null values in the dataset
train_data = train_data.fillna(0, axis =1)
train_data.isnull().sum().sum()

In [67]:
#view the statistical summary of the data
train_data.describe()

In [68]:
#let's view the columns
train_data.columns

In [69]:
#checking for columns with unique value
train_data[[col for col in train_data.columns if train_data[col].dtype == 'object']].describe()

In [70]:
#view the target
train_data['site_eui']

In [71]:
train_data['site_eui'].describe()

In [72]:
#visualize the target
train_data.hist(column='site_eui', bins = 50, figsize=(10,10))
plt.show()

As you can see that the target is positively skewed to the right showing that there are outliers. Let's go ahead and check the boundary values.

In [73]:
#check the boundary values
print("Highest allowed",train_data['site_eui'].mean() + 3 *train_data['site_eui'].std())
print("Lowest allowed",train_data['site_eui'].mean() - 3 *train_data['site_eui'].std())

Now let's find the outliers

In [74]:
#check for outliers
outliers = train_data[(train_data['site_eui'] > 257.35) | (train_data['site_eui'] > -92.18)]
outliers

Now let's trim off the outliers, cap on it and then apply the capping.

In [75]:
#trimming off the outliers
train_data =train_data[(train_data['site_eui'] < 257.35)&(train_data['site_eui'] > -92.18)]
train_data

In [76]:
#Capping on outliers
upper_limit = train_data['site_eui'].mean() + 3 *train_data['site_eui'].std()
lower_limit = train_data['site_eui'].mean() - 3 *train_data['site_eui'].std()
#Now let's apply the Capping
train_data['site_eui'] = np.where(train_data['site_eui'] > upper_limit,
                               upper_limit,np.where(train_data['site_eui'] < lower_limit,
                                                    lower_limit,train_data['site_eui']))
#reset index
train_data =train_data.reset_index(drop=True)
train_data

In [77]:
#view the summary statistics of the target
train_data['site_eui'].describe()

In [78]:
#Now let's plot the histogram again to see if the outlier has been removed.
train_data.hist(column='site_eui', bins = 50, figsize=(10,10))
plt.show()

Now if you look at the histogram, you will see that the target has been normalize. All outliers has been removed which give us similar data across all the data. Let's go ahead and check if there ia ny multicollinearity among the features and target.

In [79]:
#let's visualize the multicolinearity among the features.
sns.set(font_scale =2)
plt.figure(figsize = (60,100))
ax = sns.heatmap(train_data.corr(),mask =np.triu(np.ones_like(train_data.corr(),dtype = bool)), 
            annot = True,cmap =sns.color_palette('rocket',7),
            linewidths=2,linecolor='white',fmt='.2f',annot_kws={"size":15})
plt.title("MultiCollinearity Btw target and Features ")

In [80]:
#Using the heatmap to view the feature that correlate with the target.
plt.figure(figsize = (20,20))
ax =sns.heatmap(abs(train_data.corr())[['site_eui']], annot = True);
plt.title("Correlation between the target and features")

Looking at the graph above you can see that there is a high multi-collinearity between the target and January minimum temperature,February average temp and the cooling degree days.Now let us go ahead and split our data into dependent and independent variables

In [81]:
#define X and y
X = train_data.drop('site_eui',axis=1)
y= train_data['site_eui']
#view the shape of the data
X.shape, y.shape

In [82]:
#view the shape of the data
X.shape, y.shape

In [83]:
#View the first ten rows of the dependent features
X.head(10)

In [84]:
#view the first ten rows of the target 
y.head(10)

In [85]:
#let's view the unique values and classify them into it's unique categories
num_cols = []
ohe_cols = []
freq_cols =[]

for col in X.columns:
    if X[col].dtype in ['float64', 'int64']:
        num_cols.append(col)
    elif X[col].nunique() < 10:
        ohe_cols.append(col)
    else:
        freq_cols.append(col)
        
#print the list of numerical columns,categorical columns and frequency columns
print(f"list of numerical columns: {num_cols}")
print("--")
print(f"list of categorical columns:{ohe_cols}")
print("--")
print(f"list of Frequency columns:{freq_cols}")

It seems here that we only have two categorical variables and one freq column. Now we can go ahead split the data,and scale it so that we can have equal data among the records.

In [86]:
#split the data into training and validation data. 
#we will be using the validation data to train the algorithms
#then use the test data on the final model.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

# create another train data and validation data from the split
train_df = pd.concat([X_train,y_train], axis= 1)
validation_df = pd.concat([X_val,y_val], axis =1)

train_df.shape , validation_df.shape

In [87]:
# set up pipeline for preprocessing 
# for numeric columns, we need to scale it
# for unique value <3 columns, we need to one hot encode it
# for unique value >3 columns, we need to frequency encode it
ohe_transformer = Pipeline(steps=[
    ('ohe_imputer', SimpleImputer(strategy='constant', fill_value = 0)),
    ('oh_encoder', OneHotEncoder(handle_unknown='ignore'))
])

freq_transformer = Pipeline(steps=[
    ('freq_encoder', ce.count.CountEncoder(normalize=True, min_group_size=.05)),
    ('freq_imputer', SimpleImputer(strategy='constant', fill_value=0))
])

num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
    
])

preprocessor = ColumnTransformer(
    transformers=[
        ('ohe', ohe_transformer, ohe_cols),
        ('freq', freq_transformer, freq_cols),
        ('scaler', num_transformer, num_cols)
    ])

preprocessor.fit(X_train)

Now let's go ahead and build our baseline model so that we can use the scoring to train our data and compare across other algorithms.

# Baseline Model

In [88]:
#We will be using LinearRegression as the baseline model
baseline = Pipeline(steps = [('preprocessor', preprocessor),
                             ('reg', LinearRegression())])
baseline.fit(X_train,y_train)

In [89]:
#Use the function to evaluate the baseline model.
#evalutae the baseline model
evaluate(baseline, X_train,y_train, X_val, y_val)

As you can see the baseline model is not doing good with a R-squared score of 12%. It simply mean that the model is over-fitting compared to the number of data sample with a very high root mean square error.That said, let's go ahead and build different model and then tune them with different parameters.

# Feature Engineering with Gridsearch cv

- #### First Model with Gridsearch

In [90]:
#Initialize the DecisionTreeRegressor
d_tree = Pipeline(steps = [('preprocessor', preprocessor),
                             ('model', DecisionTreeRegressor())])
param = [{'model__criterion':['squared_error','friedman_mse'],#gridsearch using different parameters
          'model__splitter':['best'],
          'model__max_depth': [5,10,20],
          'model__min_samples_split':[10,100],
          'model__max_features': ['auto','sqrt']}]

grid = GridSearchCV(estimator= d_tree,
                    param_grid = param, scoring= 'r2')
result = grid.fit(X_train,y_train) #fit the result on train data
result.best_params_

In [91]:
#view the best score from the parameter
result.best_score_

In [92]:
#evaluate the best estimator
evaluate(result.best_estimator_,X_train,y_train, X_val, y_val)

The Rsquare is showing a variance of 15% on the training data and 15% on the validation data with a Validation RMSE OF 34.35. The model is overfitting on the variance compared to the data sample.

- #### Let's build another regressor model using gridsearch cv.

In [93]:
#Initialize the Gradient boost Regressor
gbr = Pipeline(steps = [('preprocessor', preprocessor),
                             ('gbr model',GradientBoostingRegressor())])
param = [{'gbr model__loss': ['squared_error', 'quantile'],
          'gbr model__max_features':['sqrt','auto','log2']}]

grid = GridSearchCV(estimator= gbr,
                    param_grid = param, scoring= 'r2')
result = grid.fit(X_train,y_train)
result.best_params_

In [94]:
result.best_score_

In [95]:
evaluate(result.best_estimator_,X_train,y_train, X_val, y_val)

- #### Use Final Model to predict the test data

In [96]:
#Initialize Random Forest Regressor
rf = Pipeline(steps = [('preprocessor', preprocessor),
                        ('rf model',RandomForestRegressor(criterion = 'squared_error',
                                                          max_features='auto',min_samples_split=10,n_estimators= 100))])
rf.fit(X_train,y_train)#fit the model on a train data

In [97]:
#cross validate the model
cross_validate(rf,X_train,y_train,return_train_score =True)

In [98]:
#evaluate the final model
evaluate(rf,X_train,y_train, X_val, y_val)

The Random Forest Regressor was able to outperformed the other models with a R^2 of 70% on training data and 28% on validation data with a root mean square error of 31.83. I will go ahead and use it to predict the test data.

In [104]:
#predict the model using the test data
test_data = pd.read_csv("/kaggle/input/widsdatathon2022/test.csv")#read the test data
test_data = test_data.fillna(test_data.mean()) #fill the nan value with the test data mean.
prediction = rf.predict(test_data) #prediction
df = pd.DataFrame({'Predicted': prediction}).sort_values(by ='Predicted',ascending = False)#put the prediction in a dataframe

In [105]:
#submission
submission =pd.read_csv("/kaggle/input/widsdatathon2022/sample_solution.csv")
submission['site_eui']= prediction
submission.to_csv("submission.csv",index=False)